In [1]:
from automatic_variable_mapping import corpus, vocab_similarity
import pandas as pd
from functools import partial
import time
import numpy as np
from automatic_variable_mapping.vocab_similarity import default_pairable
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Obs Heart Studies
#input_data_type = "obs_heart_studies_dbGaP_NLP"
#ref_id_col = 'dbGaP_studyID_datasetID_varID'
#doc_cols_inputs = {
#    'desc': ['variable_description'],
#    'units': ['units'],
    #'coding': ['variable_coding_counts_distribution'],
#    'desc_units': ['variable_description', 'units'],
    #'desc_coding': ['variable_description', 'units', 'variable_coding_counts_distribution'],
    #'desc_units_coding': ['variable_description', 'units', 'variable_coding_counts_distribution']
#}
#mult_doc_cols_inputs = {"desc": ['variable_description']}
#input_dir = "/Laura/tiff_laura_shared/"
#ref_file_diff = ""

In [3]:
## HF Clin Trials
#input_data_type = "HF_clin_trials_biolincc_NLP"
#ref_id_col = 'var_doc_id'
#doc_cols_inputs = {'desc': ['variable_description']}
#mult_doc_cols_inputs = {"desc": ['variable_description']}
#input_dir = "/Laura/tiff_laura_shared/"
#ref_file_diff = ""

In [4]:
# Both HF Clin Trials and Obs Heart Studies
#input_data_type = "trials_studies_NLP"
#ref_id_col = 'var_doc_id'
#doc_cols_inputs = {'desc': ['variable_description']}
#mult_doc_cols_inputs = {"desc": ['variable_description']}
#input_dir = "/Laura/tiff_laura_shared/"
#ref_file_diff = ""

In [5]:
# Both SNOMED and Obs Heart Studies
input_data_type = "obs_heart_studies_dbGaP_NLP_SNOMED_terms"
ref_id_col = 'dbGaP_studyID_datasetID_varID'
doc_cols_inputs = {'desc': ['variable_description']}
mult_doc_cols_inputs = {"desc": ['variable_description']}
input_dir = "/Laura/automatic-variable-mapping/SNOMED-concepts/output/"
man_file = input_dir + "manual_concept_var_mappings_" + "obs_heart_studies_dbGaP_NLP" + ".csv"

In [6]:
data_file = input_dir + "var_doc_" + input_data_type + ".csv"
#man_file = input_dir + "manual_concept_var_mappings_" + input_data_type + ".csv"

In [7]:
output_dir = "/data/automatic-variable-mapping-results/"
out_prefix = "var_txt_sim_scores"
num_cpus = 14

In [8]:
print data_file
print man_file

/Laura/automatic-variable-mapping/SNOMED-concepts/output/var_doc_obs_heart_studies_dbGaP_NLP_SNOMED_terms.csv
/Laura/automatic-variable-mapping/SNOMED-concepts/output/manual_concept_var_mappings_obs_heart_studies_dbGaP_NLP.csv


In [75]:
data = pd.read_csv(man_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
data.shape
#data[data["var_doc_id"] == "escape.admreas.adothr"]

(1709, 14)

In [76]:
data

,study,cohort_dbGaP,dbGaP_studyID_datasetID,dbGaP_dataset_label,concept,data_desc,varID,var_desc,units,var_coding_counts_distribution,var_doc_id,concept_id,dbGaP_studyID_datasetID_varID,var_coding_labels
0,FHS,Original,phs000007.v26.pht000009.v2,ex0_7s,Education,"Clinic Exam, Original Cohort Exams 1 - 7",MF5,EDUCATION,NaN,n=4884; nulls=195; HIGH SCHOOL GRADUATE=5:coun...,FHS1var2,education,phs000007.v26.pht000009.v2.MF5,"FIFTH, SIXTH OR SEVENTH GRADE; NONE; POST GRAD..."
1,FHS,Original,phs000007.v26.pht000009.v2,ex0_7s,number of pregnancies,"Clinic Exam, Original Cohort Exams 1 - 7",MF20,"NUMBER OF PREGNANCIES, EXAM 1",NUMBER OF PREGNANCIES,n=2122; nulls=17; mean=2.82; sd=1.353; median=...,FHS1var16,pregnancies,phs000007.v26.pht000009.v2.MF20,"MAN, SINGLE WOMAN, OR NO PREGNANCIES FOR NON-S..."
2,FHS,Original,phs000007.v26.pht000009.v2,ex0_7s,unique Participant Identifier,"Clinic Exam, Original Cohort Exams 1 - 7",shareid,UNIQUE PARTICIPANT ID,NaN,n=5079; nulls=0,FHS1var558,ParticipantID,phs000007.v26.pht000009.v2.shareid,NaN
3,FHS,Original,phs000007.v26.pht000010.v3,ex0_8s,unique Participant Identifier,"Clinic Exam, Original Cohort Exam 8",shareid,UNIQUE PARTICIPANT ID,NaN,n=3909; nulls=0,FHS2var151,ParticipantID,phs000007.v26.pht000010.v3.shareid,NaN
4,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Marital Status,"Clinic Exam, Original Cohort Exam 9",FB8,MARTIAL STATUS,NaN,n=3712; nulls=0; MARRIED=2:count=2847; WIDOWED...,FHS3var1,maritalStatus,phs000007.v26.pht000011.v3.FB8,MARRIED; SEPARATED; SINGLE; WIDOWED; DIVORCED
5,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Systolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB18,"BLOOD PRESSURE: SYSTOLIC, TAKEN BY NURSE",MM HG,n=3698; nulls=14; mean=138.7; sd=22.16; median...,FHS3var8,SBP,phs000007.v26.pht000011.v3.FB18,NaN
6,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Diastolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB19,"BLOOD PRESSURE: DIASTOLIC, TAKEN BY NURSE",MM HG,n=3699; nulls=13; mean=85.64; sd=11.88; median...,FHS3var9,DBP,phs000007.v26.pht000011.v3.FB19,NaN
7,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Systolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB20,"BLOOD PRESSURE: SYSTOLIC, PHYSICIAN'S FIRST RE...",MM HG,n=3708; nulls=4; mean=140.1; sd=23.46; median=...,FHS3var10,SBP,phs000007.v26.pht000011.v3.FB20,NaN
8,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Diastolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB21,"BLOOD PRESSURE: DIASTOLIC, PHYSICIAN'S FIRST R...",MM HG,n=3705; nulls=7; mean=82.19; sd=11.59; median=...,FHS3var11,DBP,phs000007.v26.pht000011.v3.FB21,NaN
9,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Systolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB22,"BLOOD PRESSURE: SYSTOLIC, PHYSICIAN'S SECOND R...",MM HG,n=3651; nulls=61; mean=139.1; sd=23.49; median...,FHS3var12,SBP,phs000007.v26.pht000011.v3.FB22,NaN


In [77]:
data2 = pd.read_csv(data_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
data2.shape

(133557, 15)

In [78]:
pair_ids = list(data2[data2["dbGaP_study_id"]=="SNOMED"][ref_id_col])
len(pair_ids)

27946

In [134]:
def matching_groups(data, group_col, corpora, pair_id, ref_id):
    corpus_doc_ids = [doc_id for doc_id, _ in corpora]
    ref_idx = corpus_doc_ids.index(ref_id)
    pair_idx = corpus_doc_ids.index(pair_id)
    return data[pair_idx][group_col] == data[ref_idx][group_col]


def pairable_by_group(data, group_col, corpus_doc_ids, score, pair_id, _, ref_id):
    return vocab_similarity.default_pairable(score, pair_id, None, ref_id) and not matching_groups(data, group_col, corpus_doc_ids, pair_id, ref_id)


def calc_score_results(data_file, doc_cols, doc_col_key, ref_id_col, filter_file, 
                       word_vectors=None, 
                       corpora_col=None, 
                       pair_ids_col=None):
    print data_file
    print "doc_col_key: " + doc_col_key
    data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
    
    if corpora_col:
        corpora_data = corpus.partition(data, corpora_col)
    else:
        corpora_data = [data]
    
    if filter_file != data_file:
        filter_data = pd.read_csv(filter_file,
                                  sep=",",
                                  quotechar='"',
                                  na_values="",
                                  low_memory=False)
    else:
        filter_data = data
        
    print(" Building Corpora...")
    corpora = corpus.build_corpora(doc_cols, corpora_data, ref_id_col, num_cpus=num_cpus)
    print("Corpus: ", len(corpora), " ", len([doc for c in corpora for doc in c]))
    
    print("  Calculating BOW matrix...")
    vocab, bow = corpus.calc_tfidf(corpora)
    
    # calculate embeddings
    if word_vectors:
        print("  Calculating doc embeddings matrix...")
        doc_ids, doc_vectors = corpus.calc_doc_embeddings(bow.toarray(), vocab, word_vectors, corpora)
        print("doc_ids: ", len(doc_ids), " vecs: ", len(doc_vectors))
    else:
        doc_ids = [doc for c in corpora for doc, _ in c]
        doc_vectors = bow.toarray()
    
    
    tfidf_label = "multtfcdf" if corpora_col else "tfidf"
    embed_label = "embed" if word_vectors else None
    
    out_suffix = None
    
    out_file_name = str.join('_', [i for i in [out_prefix, 
                                               input_data_type, 
                                               tfidf_label, 
                                               embed_label, 
                                               doc_col_key,
                                               out_suffix] 
                                   if i])
    out_file = output_dir + out_file_name + ".csv"
    
    ref_ids = filter_data[ref_id_col]
    
    pair_ids = None
    if pair_ids_col:
        pair_ids = filter_data[pair_ids_col]
    
    print("  Calculating Similarity Scores...")
    scores = vocab_similarity.VariableSimilarityCalculator(ref_ids, pairable=default_pairable)
    scores.init_cache(file_name=out_file)

    result = scores.score_variables(doc_ids, bow, num_cpus=num_cpus, file_name=out_file, pair_ids=pair_ids)
    
    return result

def calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, filter_file, corpora_col=None, word_vectors=None, pair_ids_col=None):
    [calc_score_results(data_file, 
                        doc_cols_inputs[key], 
                        key, 
                        ref_id_col, 
                        filter_file, 
                        corpora_col=corpora_col, 
                        word_vectors=word_vectors, 
                        pair_ids_col=pair_ids_col) 
     for key in doc_cols_inputs]

In [83]:
# bioWordVec embeddings with regular tfidf
#vec_file_dir = "/run/media/harrison/data-ntfs/WordVectors2/"
#binary_file_name = vec_file_dir + "BioWordVec_PubMed_MIMICIII_d200.vec.bin"
#biowordvec_embeddings = KeyedVectors.load_word2vec_format(binary_file_name, binary=True, limit=None)

In [51]:
#scores_embed = calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, man_file, word_vectors=biowordvec_embeddings, pair_ids_col=ref_id_col)
scores_tfidf = calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, man_file, word_vectors=None, pair_ids_col=None)
scores_tfmcdf = calc_scores_doc_cols(data_file, mult_doc_cols_inputs, ref_id_col, man_file, word_vectors=None, corpora_col='study', pair_ids_col=None)

/Laura/automatic-variable-mapping/SNOMED-concepts/output/var_doc_obs_heart_studies_dbGaP_NLP_SNOMED_terms.csv
doc_col_key: desc
 Building Corpora...


100%|██████████| 133557/133557 [00:19<00:00, 6857.77it/s] 


('Corpus: ', 1, ' ', 133557)
  Calculating BOW matrix...
  Calculating Similarity Scores...
Finding valid pair ids
Pair ids: 27946
Finding valid ref ids
Ref ids: 1709
Multiplying matrices
LHS: (27946, 42253)
RHS: (1709, 42253)
Sim Matrix: (1709, 27946)
Chunk size: 123
Finding matches for 0 phs000007.v26.pht000009.v2.MF5
Finding matches for 1600 phs000209.v13.pht002388.v1.sidno
Finding matches for 500 phs000007.v26.pht000298.v5.shareid
Finding matches for 1000 phs000287.v5.pht001486.v1.Individual_ID
Finding matches for 1500 phs000209.v13.pht001116.v10.gender1
Finding matches for 400 phs000007.v26.pht000041.v6.DIAB6
Finding matches for 900 phs000287.v5.pht001451.v1.HDL44
Finding matches for 1400 phs000280.v3.pht004193.v1.SBPB18
Finding matches for 300 phs000007.v26.pht000033.v8.D233
Finding matches for 800 phs000007.v26.pht003316.v4.chf
Finding matches for 1300 phs000280.v3.pht004126.v1.GWAS_ID
Finding matches for 700 phs000007.v26.pht002236.v4.shareid
Finding matches for 200 phs000007.v

100%|██████████| 1709/1709 [00:00<00:00, 1256673.48it/s]


/Laura/automatic-variable-mapping/SNOMED-concepts/output/var_doc_obs_heart_studies_dbGaP_NLP_SNOMED_terms.csv
doc_col_key: desc
 Building Corpora...


100%|██████████| 22147/22147 [00:15<00:00, 1465.27it/s]
0it [00:00, ?it/s]


('Corpus: ', 5, ' ', 105611)
  Calculating BOW matrix...


ValueError: Found array with 0 sample(s) (shape=(0, 33026)) while a minimum of 1 is required.

## Top N words from concept vectors

In [135]:
from scipy.sparse import csr_matrix

def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.hstack((a.data,b.data))
    a.indices = np.hstack((a.indices,b.indices))
    a.indptr = np.hstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

def calc_top_n_words(data_file, doc_cols, doc_col_key, ref_id_col, filter_file, n, word_vectors=None, corpora_col=None, pair_ids_col=None):
    print data_file
    print "doc_col_key: " + doc_col_key
    data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
    
    if corpora_col:
        corpora_data = corpus.partition(data, corpora_col)
    else:
        corpora_data = [data]
    
    if filter_file != data_file:
        filter_data = pd.read_csv(filter_file,
                                  sep=",",
                                  quotechar='"',
                                  na_values="",
                                  low_memory=False)
    else:
        filter_data = data
        
    print(" Building Corpora...")
    corpora = corpus.build_corpora(doc_cols, corpora_data, ref_id_col, num_cpus=num_cpus)
    print("Corpus: ", len(corpora), " ", len([doc for c in corpora for doc in c]))
    
    print("  Calculating BOW matrix...")
    vocab, bow = corpus.calc_tfidf(corpora)
    
    # calculate embeddings
    if word_vectors:
        print("  Calculating doc embeddings matrix...")
        doc_ids, doc_vectors = corpus.calc_doc_embeddings(bow.toarray(), vocab, word_vectors, corpora)
        print("doc_ids: ", len(doc_ids), " vecs: ", len(doc_vectors))
    else:
        doc_ids = [doc for c in corpora for doc, _ in c]
        doc_vectors = bow.toarray()
    
    
    tfidf_label = "multtfcdf" if corpora_col else "tfidf"
    embed_label = "embed" if word_vectors else None
    
    out_suffix = "withConcepts"
    
    out_file_name = str.join('_', [i for i in [out_prefix, 
                                               input_data_type, 
                                               tfidf_label, 
                                               embed_label, 
                                               doc_col_key,
                                               out_suffix] 
                                   if i])
    out_file = output_dir + out_file_name + ".csv"
    
    print("  Calculating Top N words..")
    filter_groups = filter_data.groupby(["concept"])[ref_id_col]
    groups = {group: data[ref_id_col].isin(filter_groups.get_group(group)) for group in list(filter_groups.groups)}
    normalzed_doc_vectors_by_group = corpus.normalize_doc_vectors_by_group(doc_vectors, groups)
    top_words_in_group = {group: corpus.vec_top_words(vocab, vec, n) for group, vec in normalzed_doc_vectors_by_group.items()}

    ref_ids = normalzed_doc_vectors_by_group.keys() 
    pair_ids = None
    if pair_ids_col:
        pair_ids = filter_data[pair_ids_col]
    
    print("  Calculating Similarity Scores...")
    scores = vocab_similarity.VariableSimilarityCalculator(ref_ids, pairable=default_pairable)
    scores.init_cache(file_name=out_file)

    normalzed_doc_vectors_matrix = csr_matrix(np.vstack(normalzed_doc_vectors_by_group.values()))
    bow_with_concepts = csr_vappend(bow, normalzed_doc_vectors_matrix)
    doc_ids_with_concepts = doc_ids + normalzed_doc_vectors_by_group.keys()

    scores.score_variables(doc_ids_with_concepts, 
                           bow_with_concepts, 
                           num_cpus=num_cpus, 
                           file_name=out_file, 
                           pair_ids=pair_ids)
    
    
    return top_words_in_group

def calc_top_n_words_by_doc_cols(data_file, doc_cols_inputs, ref_id_col, filter_file, n, corpora_col=None, word_vectors=None, pair_ids_col=None):
    return {key: calc_top_n_words(data_file, 
                        doc_cols_inputs[key], 
                        key, 
                        ref_id_col, 
                        filter_file, 
                        n,
                        corpora_col=corpora_col, 
                        word_vectors=word_vectors, 
                        pair_ids_col=pair_ids_col)
                for key in doc_cols_inputs}

In [136]:
n = 5
top_n_words = calc_top_n_words_by_doc_cols(data_file, 
                                            doc_cols_inputs, 
                                            ref_id_col, 
                                            man_file, 
                                            n,
                                            word_vectors=None, 
                                            pair_ids_col=ref_id_col)
top_n_words

/Laura/automatic-variable-mapping/SNOMED-concepts/output/var_doc_obs_heart_studies_dbGaP_NLP_SNOMED_terms.csv
doc_col_key: desc
 Building Corpora...


100%|██████████| 133557/133557 [00:18<00:00, 7263.85it/s] 


('Corpus: ', 1, ' ', 133557)
  Calculating BOW matrix...
  Calculating Top N words..
  Calculating Similarity Scores...
Finding valid pair ids
Pair ids: 1709
Finding valid ref ids
Ref ids: 54
Multiplying matrices
LHS: (1709, 42253)
RHS: (54, 42253)
Sim Matrix: (54, 1709)
Chunk size: 4
Finding matches for 0 HDL cholesterol mmol_L


100%|██████████| 54/54 [00:00<00:00, 32374.56it/s]


{'desc': {'AF': ['atrial',
   'fibrillation',
   'diagnosis',
   'determined',
   '31decyy'],
  'Age (years)': ['age', 'exam', 'calculated', 'visit', 'yr'],
  'Atrial Fibrillation Status (A-fib/flutter)': ['atrial',
   'fibrillation',
   'flutter',
   'ecg',
   'fib'],
  'Average Diastolic Blood Pressure (mmHg)': ['pressure',
   'diastolic',
   'seated',
   'mmhg',
   'blood'],
  'Average Systolic Blood Pressure (mmHg)': ['pressure',
   'seated',
   'mmhg',
   'systolic',
   'blood'],
  'BMI (kg/m^2)': ['mass', 'index', 'body', 'bmi', 'kg'],
  'CHD': ['chd', 'coronary', 'incident', 'disease', 'smi'],
  'CHDDeath': ['chd', 'fatchd', 'fatal', 'coronary', '2007'],
  'CHF': ['chf', 'congestive', 'incident', 'failure', 'heart'],
  'CVD': ['cvd', 'cardiovascular', 'disease', 'status'],
  'CVDDeath': ['cvd', 'nonfatal', 'fatal', 'cardiovascular', 'hard'],
  'Caffine Intake (mg)': ['caffeine', 'mg', 'nutrient', 'field', 'derived'],
  'Cholesterol Lowering Medication': ['lowering',
   'lipid',


In [112]:
doc_col_key = "desc"  
filter_file = man_file
corpora_col = None
key = "desc"
doc_cols = doc_cols_inputs[key]
pair_ids_col = ref_id_col

In [129]:
reload(corpus)
reload(vocab_similarity)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<module 'automatic_variable_mapping.vocab_similarity' from 'automatic_variable_mapping/vocab_similarity.pyc'>

(133719, 42253)
(54, 42253)
(133773, 42253)
Finding valid pair ids
Pair ids: 1709
Finding valid ref ids
Ref ids: 54
Multiplying matrices
LHS: (1709, 42253)
RHS: (54, 42253)
Sim Matrix: (54, 1709)
Chunk size: 4
Finding matches for 0 HDL cholesterol mmol_L


100%|██████████| 54/54 [00:00<00:00, 25973.90it/s]
